## Run once:
```
activate py35 && pip install colorlover
```

# Introduction to NetworkX

In [ ]:
import networkx as nx

## Creating a graph

In [ ]:
G = nx.Graph()

In [ ]:
G.add_node(234)
G.add_node("hello")
G.add_edge(234,"hello")

In [ ]:
print("Nodes:", G.nodes())
print("Edges:", G.edges())

In [ ]:
G[234]

In [ ]:
G['hello']

## Properties on edges

In [ ]:
G.add_edge('Alice', 'Bob', {'know': 10, 'friends': 5})

In [ ]:
print("Nodes:", G.nodes())
print("Edges:", G.edges())

In [ ]:
G['Bob']

In [ ]:
G['Alice']

In [ ]:
G['Bob']['Alice']['know'] += 1

In [ ]:
G['Alice']

## Analyzing graphs

In [ ]:
G.add_edge('Alice', 'Carlos')
G.add_edge('Carlos', 'Dave')
G.add_edge('Dave', 'Bob')
G.add_edge('Alice', 'Eve')

In [ ]:
components = nx.connected_components(G)
list(components)

In [ ]:
nx.degree(G)

In [ ]:
nx.degree(G,'Bob')

In [ ]:
nx.has_path(G, 'Alice', 'Dave')

In [ ]:
nx.has_path(G, 'Alice', 'hello')

In [ ]:
nx.shortest_path(G, 'Alice', 'Dave')

## Centrality

In [ ]:
nx.degree_centrality(G)

In [ ]:
nx.betweenness_centrality(G)

# Loading Twitter Data

In [ ]:
import pandas as pd

In [ ]:
!pwd

### Download the tweets CSV from https://goo.gl/NbpSek
### Put "tweets2009-06-0115.csv.zip" in the current folder (based on "!pwd" above)

In [ ]:
df = pd.read_csv("tweets2009-06-0115.csv.zip", sep='\t', compression='zip')

In [ ]:
print("Num of rows:", df.shape[0])

In [ ]:
df.head()

## Finding most common hash tags

In [ ]:
from collections import Counter

In [ ]:
allTweets = df["tweet"].str.cat(sep=' ')
tweetWords = [word.strip(""" ,.:'\";""").lower() for word in allTweets.split()]
hashTags = [word for word in tweetWords if word.startswith("#")]
hashTagsCounter = Counter(hashTags)

In [ ]:
hashTagsCounter.most_common(100)

In [ ]:
redsoxTag = df[df["tweet"].str.lower().str.contains("#redsox", na=False)].copy()

In [ ]:
redsoxTag.head()

In [ ]:
def addMentionedColumn(df):
    
    def mentionsList(txt):
        allWords = [word.strip(""" ,.:'\";""").lower() for word in txt.split()]
        allNames = [word.strip("@") for word in allWords if word.startswith("@")]
        uniqueNames = list(set(allNames))
        return allNames
    
    df["mentioned"] = df["tweet"].apply(mentionsList)

In [ ]:
addMentionedColumn(redsoxTag)

In [ ]:
redsoxTag.head(10)

In [ ]:
def mentionGraph(df):
    g = nx.Graph()
    
    for (index, date, user, tweet, mentionedUsers) in df.itertuples():
        for mentionedUser in mentionedUsers:
            if (user in g) and (mentionedUser in g[user]):
                g[user][mentionedUser]["numberMentions"] += 1
            else:
                g.add_edge(user, mentionedUser, {'numberMentions': 1})
    
    return g

In [ ]:
redsoxGraph = mentionGraph(redsoxTag)

In [ ]:
print("# nodes:", len(redsoxGraph.nodes()))
print("# edges:", len(redsoxGraph.edges()))

In [ ]:
redsoxGraph['shelley1005']

# Visualize Mention Graph

In [ ]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True)

### Generate random positions for nodes and store them at property "pos"

In [ ]:
import random
def addRandomPositions(graph):
    posDict = dict((node,(random.gauss(0,10),random.gauss(0,10))) for node in graph.nodes())
    nx.set_node_attributes(graph,"pos", posDict)

In [ ]:
addRandomPositions(redsoxGraph)

In [ ]:
nx.get_node_attributes(redsoxGraph, 'pos')['shelley1005']

### Visualize using Plot.ly scatter plots

In [ ]:
def plotNetwork(graph):
    scatters=[]

    for (node1, node2) in graph.edges():
        x0, y0 = graph.node[node1]['pos']
        x1, y1 = graph.node[node2]['pos']
        edgeWidth = graph[node1][node2]['numberMentions']
        s = Scatter(
                x=[x0, x1],
                y=[y0, y1],
                hoverinfo='none',
                mode='lines', 
                line=Line(width=edgeWidth ,color='#888'))
        scatters.append(s)



    for node in graph.nodes():
        xPos, yPos = graph.node[node]['pos']
        s = Scatter(
                x=[xPos], 
                y=[yPos], 
                hoverinfo='none',
                mode='marker', 
                marker=dict(
                    color="#888", 
                    size=10,         
                    line=dict(width=2)))
        scatters.append(s)
    
    layout = Layout(showlegend=False)
    fig = Figure(data=scatters, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
plotNetwork(redsoxGraph)

### Visualize using node size and edge width

In [ ]:
def plotNetworkSize(graph):
    scatters=[]

    for (node1, node2) in graph.edges():
        x0, y0 = graph.node[node1]['pos']
        x1, y1 = graph.node[node2]['pos']
        edgeWidth = graph[node1][node2]['numberMentions']
        s = Scatter(
                x=[x0, x1],
                y=[y0, y1],
                hoverinfo='none',
                mode='lines', 
                line=Line(width=edgeWidth ,color='#888'))
        scatters.append(s)



    for node in graph.nodes():
        xPos, yPos = graph.node[node]['pos']
        s = Scatter(
                x=[xPos], 
                y=[yPos], 
                hoverinfo='none',
                mode='marker', 
                marker=dict(
                    color="#888", 
                    size=nx.degree(graph,node)*2,         
                    line=dict(width=2)))
        scatters.append(s)
    
    layout = Layout(showlegend=False)
    fig = Figure(data=scatters, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
plotNetworkSize(redsoxGraph)

## Using *Colorlover* for colors

In [ ]:
import colorlover as cl
from IPython.display import HTML

In [ ]:
HTML(cl.to_html( cl.scales['9'] ))

In [ ]:
# map purd color scale to 300 cells
purd = cl.scales['9']['seq']['PuRd']
purd300 = cl.interp(purd, 300)
HTML(cl.to_html(purd300))

## Adding color and text based on centrality

In [ ]:
def plotNetworkSizeColor(graph):
    closenessCentr = nx.closeness_centrality(redsoxGraph)
    maxCentr = max(closenessCentr.values())
    minCentr = min(closenessCentr.values())
    
    scatters=[]

    for (node1, node2) in graph.edges():
        x0, y0 = graph.node[node1]['pos']
        x1, y1 = graph.node[node2]['pos']
        edgeWidth = graph[node1][node2]['numberMentions']
        s = Scatter(
                x=[x0, x1],
                y=[y0, y1],
                hoverinfo='none',
                mode='lines', 
                line=Line(width=edgeWidth ,color='#888'))
        scatters.append(s)



    for node in graph.nodes():
        nodeCentr = closenessCentr[node]
        nodeColor = int(299*(nodeCentr-minCentr)/(maxCentr-minCentr))
        xPos, yPos = graph.node[node]['pos']
        s = Scatter(
                x=[xPos], 
                y=[yPos], 
                text="User: %s\nCloseness: %.3f" % (node, nodeCentr),
                hoverinfo='text',
                mode='marker', 
                marker=dict(
                    color=purd300[nodeColor], 
                    size=nx.degree(graph,node)*2,         
                    line=dict(width=2)))
        scatters.append(s)
    
    layout = Layout(showlegend=False)
    fig = Figure(data=scatters, layout=layout)
    iplot(fig, show_link=False)

In [ ]:
plotNetworkSizeColor(redsoxGraph)

## Using NetworkX layouts

In [ ]:
def applyLayout(graph, layoutFunc):
    posDict = layoutFunc(graph) 
    nx.set_node_attributes(graph, "pos", posDict)

### Spring layout

In [ ]:
redsoxGraphSpring = redsoxGraph.copy()
applyLayout(redsoxGraphSpring, nx.spring_layout)
plotNetworkSizeColor(redsoxGraphSpring)

### Random layout

In [ ]:
redsoxGraphRandom = redsoxGraph.copy()
applyLayout(redsoxGraphRandom, nx.random_layout)
plotNetworkSizeColor(redsoxGraphRandom)

### Circular layout

In [ ]:
redsoxGraphCircular = redsoxGraph.copy()
applyLayout(redsoxGraphCircular, nx.circular_layout)
plotNetworkSizeColor(redsoxGraphCircular)

### Spectral layout

In [ ]:
redsoxGraphSpectral = redsoxGraph.copy()
applyLayout(redsoxGraphSpectral, nx.spectral_layout)
plotNetworkSizeColor(redsoxGraphSpectral)